<h2>Import functions and libraries:

In [1]:
from gurobipy import *
import pandas as pd

# Sorting fuctions
def B_bubbleSort(b, job):
    for i in N:
        for j in range(0, n-i-1):
            if b[job[j]] < b[job[j+1]]:
                job[j], job[j+1] = job[j+1], job[j]
                
def R_bubbleSort(b, job):
    for i in N:
        for j in range(0, n-i-1):
            if (b[job[j]]/p[job[j]]) < (b[job[j+1]]/p[job[j+1]]):
                job[j], job[j+1] = job[j+1], job[j]
                
def RB_bubbleSort(b, job):
    for i in N:
        for j in range(0, n-i-1):
            if (b[job[j]]/p[job[j]]) < (b[job[j+1]]/p[job[j+1]]):
                job[j], job[j+1] = job[j+1], job[j]        
    for i in N:
        for j in range(0, n-i-1):
            if (b[job[j]]/p[job[j]]) == (b[job[j+1]]/p[job[j+1]]) and (b[job[j]] < b[job[j+1]]):
                job[j], job[j+1] = job[j+1], job[j]
                
def M_bubbleSort(bm, machine):
    for i in M:
        for j in range(0, m-i-1):
            if bm[machine[j]] > bm[machine[j+1]]:
                machine[j], machine[j+1] = machine[j+1], machine[j]

<h2>Input Data:

In [2]:
# Input

'''
# numeric
#(b)
m = 3
n = 10
b = [5, 8, 4, 6, 3, 7, 6, 9, 5, 8]
p = [3, 6, 5, 4, 1, 8, 5, 12, 7, 6]
K = 15

N = range(n)
M = range(m)
'''


#(c)(d)
m = 4
n = 25
b = [5,8,12,4,6,6,3,7,6,15,9,5,8,10,1,5,3,7,12,14,5,8,9,8,10]
p = [3,6,7,5,4,2,6,3,5,8,10,2,4,7,1,5,8,3,6,4,12,4,8,4,7]
K = 15

N = range(n)
M = range(m)


'''
# 4.
# file input
f = open('in01.txt', 'r')
line1 = f.readline()
line2 = f.readline()
line3 = f.readline()

array = line1.split( )
array = list(map(int, array))
    
m = array[0]
n = array[1]
K = array[2]

b = line2.split( )
b = list(map(int, b))

p = line3.split( )
p = list(map(int, p))

N = range(n)
M = range(m)
f.close()
'''

"\n# 4.\n# file input\nf = open('in01.txt', 'r')\nline1 = f.readline()\nline2 = f.readline()\nline3 = f.readline()\n\narray = line1.split( )\narray = list(map(int, array))\n    \nm = array[0]\nn = array[1]\nK = array[2]\n\nb = line2.split( )\nb = list(map(int, b))\n\np = line3.split( )\np = list(map(int, p))\n\nN = range(n)\nM = range(m)\nf.close()\n"

<h2>(b) Linear Relaxation:

In [3]:
# Modeling
# add var
p3_1 = Model("problem3-1")    # build a new model

x = p3_1.addVars(M, N, lb = 0, ub = 1, vtype = GRB.CONTINUOUS, name = "x")
y = p3_1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "y")

# setting the objective function 
p3_1.setObjective(
    y
    , GRB.MAXIMIZE) 

# add constraints and name them
for i in M:
    p3_1.addConstr(y <= (quicksum(x[i, j] * b[j] for j in N)), name = f"min benefit")
    
for j in N:
    p3_1.addConstr(quicksum(x[i, j] for i in M) <= 1, name = f"distribute to no more than 1 machine")
    
for i in M:
    p3_1.addConstr(quicksum(x[i, j] * p[j] for j in N) <= K, name = f"satisfy capacity")
    
p3_1.optimize()

# Results
print("Results:\n")

# objective value
LR_ov = p3_1.objVal
print("objective value =", LR_ov)
print("")
    
# x
print("x:")
print("J \ M", end="")
print("\tMachine1 \tMachine2 \tMachine3")
for j in N:
    # mark which product is printed now
    print("Job" + str(j+1), "\t", end="")
    for i in M:
        print(round((x[i, j].x), 4), "\t\t", end="")
    print("")    # use for change line
print("")

# y
print("y:")
print(y.x)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 33 rows, 101 columns and 304 nonzeros
Model fingerprint: 0x3a862f1c
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Presolved: 33 rows, 101 columns, 304 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   4.000000e+30   8.000000e+00      0s
      67    3.0392857e+01   0.000000e+00   0.000000e+00      0s

Solved in 67 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.039285714e+01
Results:

objective value = 30.392857142857146

x:
J \ M	Machine1 	Machine2 	Machine3
Job1 	0.0 		1.0 		0.0 		0.0 		
Job2 	0.0 		0.0 		0.0 		0.0 		
Job3 	0.0 		

<h2>(c) HBF heuristic algorithm:

In [4]:
# Solve
# Step 1: Sorting
job_sorted = []
for i in N:
    job_sorted.append(i)

B_bubbleSort(b, job_sorted)
'''
for i in job_sorted:
    print(i, b[i])
print("")
'''
    
# Step 2: Scheduling
job_to_machine = [0] * n
Pm = [0] * m
Bm = [0] * m

for i in job_sorted:
    # sort machine in priority
    machine_sorted = []
    for k in M:
        machine_sorted.append(k)
    M_bubbleSort(Bm, machine_sorted)
    
    # schedule jods to machine
    scheduled = False
    for j in machine_sorted:
        if (Pm[j] + p[i] <= K):
            job_to_machine[i] = j
            Pm[j] += p[i]
            Bm[j] += b[i]
            scheduled = True
            break         
    if (scheduled == False):
        job_to_machine[i] = -1
        
# print results 
for i in job_sorted:
    print("job", i + 1, ", machine", job_to_machine[i] + 1)
display("Accumulated processing time:", Pm)
display("Accumulated benefits:", Bm)
HBF_ov = min(Bm)
print("objective value:", HBF_ov)

job 10 , machine 1
job 20 , machine 2
job 3 , machine 3
job 19 , machine 4
job 14 , machine 3
job 25 , machine 4
job 11 , machine 2
job 23 , machine 0
job 2 , machine 1
job 13 , machine 0
job 22 , machine 0
job 24 , machine 0
job 8 , machine 0
job 18 , machine 0
job 5 , machine 0
job 6 , machine 4
job 9 , machine 0
job 1 , machine 0
job 12 , machine 0
job 16 , machine 0
job 21 , machine 0
job 4 , machine 0
job 7 , machine 0
job 17 , machine 0
job 15 , machine 3


'Accumulated processing time:'

[14, 14, 15, 15]

'Accumulated benefits:'

[23, 23, 23, 28]

objective value: 23


<h2>(d) HRF heuristic algorithm:

In [5]:
# Solve
# Step 1: Sorting
job_sorted = []
for i in N:
    job_sorted.append(i)

R_bubbleSort(b, job_sorted)
'''
for i in job_sorted:
    print(i, b[i])
print("")
'''
    
# Step 2: Scheduling
job_to_machine = [0] * n
Pm = [0] * m
Bm = [0] * m

for i in job_sorted:
    # sort machine in priority
    machine_sorted = []
    for k in M:
        machine_sorted.append(k)
    M_bubbleSort(Bm, machine_sorted)
    
    # schedule jods to machine
    scheduled = False
    for j in machine_sorted:
        if (Pm[j] + p[i] <= K):
            job_to_machine[i] = j
            Pm[j] += p[i]
            Bm[j] += b[i]
            scheduled = True
            break         
    if (scheduled == False):
        job_to_machine[i] = -1
        
# print results 
for i in job_sorted:
    print("job", i + 1, ", machine", job_to_machine[i] + 1)
display("Accumulated processing time:", Pm)
display("Accumulated benefits:", Bm)
HRF_ov = min(Bm)
print("objective value:", HRF_ov)

job 20 , machine 1
job 6 , machine 2
job 12 , machine 3
job 8 , machine 4
job 18 , machine 3
job 13 , machine 2
job 19 , machine 4
job 22 , machine 3
job 24 , machine 1
job 10 , machine 2
job 3 , machine 1
job 1 , machine 4
job 5 , machine 3
job 14 , machine 0
job 25 , machine 0
job 2 , machine 0
job 9 , machine 0
job 23 , machine 0
job 15 , machine 4
job 16 , machine 0
job 11 , machine 0
job 4 , machine 0
job 7 , machine 0
job 21 , machine 0
job 17 , machine 0


'Accumulated processing time:'

[15, 14, 13, 13]

'Accumulated benefits:'

[34, 29, 26, 25]

objective value: 25


<h2>Percentage optimality gap:

In [6]:
print("HBF optimality gap:", (LR_ov - HBF_ov) / LR_ov)
print("HRF optimality gap:", (LR_ov - HRF_ov) / LR_ov)

HBF optimality gap: 0.2432432432432433
HRF optimality gap: 0.17743830787309056


<h2>Self design heuristic algorithm:

In [7]:
# Solve
# Step 1: Sorting
job_sorted = []
for i in N:
    job_sorted.append(i)

RB_bubbleSort(b, job_sorted)
'''
for i in job_sorted:
    print(i, b[i])
print("")
'''
    
# Step 2: Scheduling
job_to_machine = [0] * n
Pm = [0] * m
Bm = [0] * m

for i in job_sorted:
    # sort machine in priority
    machine_sorted = []
    for k in M:
        machine_sorted.append(k)
    M_bubbleSort(Bm, machine_sorted)
    
    # schedule jods to machine
    scheduled = False
    for j in machine_sorted:
        if (Pm[j] + p[i] <= K):
            job_to_machine[i] = j
            Pm[j] += p[i]
            Bm[j] += b[i]
            scheduled = True
            break         
    if (scheduled == False):
        job_to_machine[i] = -1
        
# print results 
for i in job_sorted:
    print("job", i + 1, ", machine", job_to_machine[i] + 1)
display("Accumulated processing time:", Pm)
display("Accumulated benefits:", Bm)
HRBF_ov = min(Bm)
print("objective value:", HRBF_ov)

job 20 , machine 1
job 6 , machine 2
job 12 , machine 3
job 8 , machine 4
job 18 , machine 3
job 19 , machine 2
job 13 , machine 4
job 22 , machine 3
job 24 , machine 1
job 10 , machine 4
job 3 , machine 2
job 1 , machine 3
job 5 , machine 1
job 14 , machine 0
job 25 , machine 0
job 2 , machine 0
job 9 , machine 0
job 23 , machine 0
job 16 , machine 0
job 15 , machine 3
job 11 , machine 0
job 4 , machine 0
job 7 , machine 0
job 21 , machine 0
job 17 , machine 0


'Accumulated processing time:'

[12, 15, 13, 15]

'Accumulated benefits:'

[28, 30, 26, 30]

objective value: 26


<h2>Percentage optimality gap:

In [8]:
print("HRBF optimality gap:", (LR_ov - HRBF_ov) / LR_ov)

HRBF optimality gap: 0.1445358401880142
